In [4]:
import matplotlib as mpl
from matplotlib.pyplot import *
#from fm import *
import time
import pickle
from scipy import constants
import numpy as np
from matplotlib.pyplot import cm
import pdb
from numba import jit
import math
import numpy as np
import scipy as sp
from array import *
from scipy import interpolate
from scipy import special
from scipy import interp
from scipy import ndimage
from astropy.io import fits
import pdb
import datetime
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
import matplotlib.pyplot as plt

In [3]:
def airtovac(wlnm):
    #Convert wavelengths (nm) in air to wavelengths in vaccuum (empirical).
    wlA = wlnm*10.0
    s = 1e4 / wlA
    n = 1 + (0.00008336624212083 + 0.02408926869968 / (130.1065924522 - s**2) +
    0.0001599740894897 / (38.92568793293 - s**2))
    return wlA * n / 10.0
data_dir = './reference_data/observations/WASP-121b'
import h5py
import numpy as np
with h5py.File(data_dir+'/MyFinalExt_WASP-121_t1_2016-12-26.h5', 'r') as F:
    # Note that doppler.Eldin_AirToVacuum converts from air to vacuum wavelength - let me know if you want the code.
    wl_grid = airtovac(F['blaze/blue/wl'][:])

In [2]:
data_dir = './reference_data/observations/WASP-121b'
al = np.load(data_dir+'/aligned_C_arrays.npz')
wl_grid, _ = pickle.load(open(data_dir+'/PCA_matrix.pic', 'rb'))
data, Ce = al['aligned/blue/C'], al['aligned/blue/Ce'] # etc, replace blue with redl/redu
Ndet, Nphi, Npix = data.shape #yup, this again--Norders x Nphases x Npixels

In [3]:
data_norm = np.zeros(data.shape)
for i in range(len(data)):
    order = data[i]
    order_norm = (order.T / np.median(order, axis=1)).T
    data_norm[i] = order_norm
data = data_norm

In [4]:
#SVD/PCA method-----------------------------------

data_arr1=np.zeros(data.shape)
data_arr=np.zeros(data.shape)

NPC=4 #change the "4" to whatever. This is the number of PC's to remove

for i in range(Ndet):
    #taking only first four vectors, reconstructiong, and saving
    u,s,vh=np.linalg.svd(data[i,],full_matrices=False)  #decompose
    s[NPC:]=0.  
    W=np.diag(s)
    A=np.dot(u,np.dot(W,vh))
    data_arr1[i,]=A

    #removing first four vectors...this is the 'processesed data'
    u,s,vh=np.linalg.svd(data[i,],full_matrices=False)  #decompose--not sure why I did it again....guess you don't really need this line
    s[0:NPC]=0.
    W=np.diag(s)
    A=np.dot(u,np.dot(W,vh))
    #sigma clipping sort of--it really doesn't make a yuge difference.
    #'''
    sig=np.std(A)
    med=np.median(A)
    loc=np.where(A > 3.*sig+med)
    A[loc]=0#*0.+20*sig
    loc=np.where(A < -3.*sig+med)
    A[loc]=0#*0.+20*sig
    
    data_arr[i,]=A

In [5]:
#saving pickles--these are used in the retrieval/log-likelihood/CCF analysis
pickle.dump([wl_grid,data_arr1],open(data_dir+'/data_to_scale_with.pic','wb')) #this is all the telluric crud you removed. Will be used for model injection
pickle.dump([wl_grid,data_arr],open(data_dir+'/PCA_matrix.pic','wb'))  #this is the "PCA'd" cube. It just looks like noise. The planet is in there somewhere...; wavelength might need /1000 if in A

In [6]:
import dill
pl_dat = dill.load(open('{}/planet_info_py3.dill'.format(data_dir),'rb'))
p_vel1 = pl_dat['p_vel1']   # planet velocity for transit 1 (transit 2 exists but is low quality)
tmodel1 = pl_dat['tmodel1'] # transit model, used for weighting
bv1 = pl_dat['bv1']         # barycentric correction
ph1 = pl_dat['ph1']         # orbital phase


pickle.dump(bv1, open(data_dir+'/rvel.pic','wb'))  #the Vbary+Vsys now saved as one happy array pickle
pickle.dump(ph1, open(data_dir+'/ph.pic','wb'))  #the Vbary+Vsys now saved as one happy array pickle

In [9]:
from POSEIDON.high_res import make_data_cube
data_dir = './reference_data/observations/WASP-77Ab_injection_test'
wl_grid, data = pickle.load(open(data_dir+'/data_RAW.pic', 'rb'))
print(data.shape)
data_scale, data_arr = make_data_cube(data, wl_grid)

#saving pickles--these are used in the retrieval/log-likelihood/CCF analysis
pickle.dump([wl_grid, data_scale],open(data_dir+'/data_to_scale_with.pic','wb')) #this is all the telluric crud you removed. Will be used for model injection
pickle.dump([wl_grid, data_arr],open(data_dir+'/PCA_matrix.pic','wb'))  #this is the "PCA'd" cube. It just looks like noise. The planet is in there somewhere...; wavelength might need /1000 if in A

(44, 79, 1848)
